In [0]:
import os
import tensorflow as tf

import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import shutil

import keras
import keras.backend as K
from keras.models import Model
from keras import backend as K
from keras.utils import np_utils
from keras.optimizers import SGD, Adam
from keras.datasets import cifar10
from keras.layers.core import Layer
from keras.models import model_from_json
from keras.callbacks import LearningRateScheduler
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D,Flatten

import cv2 
import numpy as np 
import math 

In [0]:
dest = "/content/drive/My Drive/GRID" 
#Note: The GRID folder here contains another GRID folder inside which has all the images in a images folder
#Example: "/content/drive/My Drive/GRID/GRID/images/JPEG_20161128_141221_1000743655515.png"

os.chdir(dest)
print('\nCurrent...')
!ls

In [0]:
#Generate Path and labels to Training Data

df = pd.read_csv('training_set.csv')
df = shuffle(df, random_state=42)
df['x1'] = df['x1']/640
df['x2'] = df['x2']/640
df['y1'] = df['y1']/480
df['y2'] = df['y2']/480
df['xc'] = (df['x1']+df['x2'])/2
df['yc'] = (df['y1']+df['y2'])/2
df['w'] = (df['x2']-df['x1'])
df['h'] = (df['y2']-df['y1'])

trainfileslist = df['image_name'].tolist()
trainfileslist = ["GRID/images/" + s for s in trainfileslist]
trainfileslist = [os.path.abspath(s) for s in trainfileslist ]
print('Path.... =>', trainfileslist[10])

coordinates = df[['xc', 'yc', 'w', 'h']].values

#Plot Image with Bounding Box
def plot_image(i):
    img = Image.open(trainfileslist[i])
    fig,ax = plt.subplots(1)
    ax.imshow(img)
    x = (coordinates[i,0]-(coordinates[i,2]/2))*640
    y = (coordinates[i,1]-(coordinates[i,3]/2))*480
    w = (coordinates[i,2])*640
    h = (coordinates[i,3])*480
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    plt.show()


Path.... => /content/drive/My Drive/GRID/GRID/images/JPEG_20161128_141221_1000743655515.png


In [0]:
plot_image(9999)

In [0]:
#save train images in a separate folder (train_file)

def save_train():
    dest = "/content/drive/My Drive/GRID/GRID/train_file/"    
    os.chdir("../..")
    for file_name in trainfileslist:
        if (os.path.isfile(file_name)):
            shutil.copy(file_name, dest)

save_train()

In [0]:
#Develop the Architecture and compile as a Keras model

def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output
  
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
#x1 = Dropout(rate = 0.1)(x1)
x1 = Dense(2, activation='sigmoid', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
#x2 = Dropout(rate = 0.1)(x2)
x2 = Dense(2, activation='sigmoid', name='auxilliary_output_2')(x2)

#save the Architecture as model and compile it appropriate losses
model = Model(input_layer, [x1, x2], name='inception_v1')
model.summary()

initial_lrate = 0.025
sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='mean_absolute_error',  optimizer=sgd)

# Function to load Training Data in Batches
def load_batch_data(index, minibatchsize):
    images = []
    batchlist = trainfileslist[index*minibatchsize:(index+1)*minibatchsize]
    for i,f in enumerate(batchlist):
        image = Image.open(f)
        image = image.resize((224, 224))
        image = np.array(image)
        images.append(image)
    images = np.array(images)
    
    X_train = images.astype('float32')
    X_train = (X_train) / 255.0
    
    Y_train = coordinates[index*minibatchsize:(index+1)*minibatchsize,:]
    Y_train = Y_train.astype('float32')
    
    return X_train, Y_train

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 56, 56, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 56, 56, 64)   4160        max_pool_1_3x3/2[0][0]           
_____________________________________

In [0]:
# Load model and compile if already available

json_file = open('model.json', 'r')
model = json_file.read()
json_file.close()
model = model_from_json(model)
model.load_weights('/content/drive/My Drive/GRID/model.h5')
model.compile(loss='mean_absolute_error',  optimizer=sgd)

In [0]:
#Train the Model!

minibatchsize = 250  

def train(i):
    print("start")
    X_train, Y_train = load_batch_data(i, minibatchsize)
    print('end')
    history = model.train_on_batch(X_train, [Y_train[:,:2],Y_train[:,2:]])
    score = model.evaluate(x=X_train, y=[Y_train[:,:2],Y_train[:,2:]], batch_size=None, verbose=1, sample_weight=None, steps=None)
    print('Epoch: ', j, 'Iter: ', i, "Metric: ", score)
    if i%10 == 5:
        model_json = model.to_json()
        with open("model.json", "w") as json_file:
            json_file.write(model_json)
        model.save_weights("model.h5")
        print("Saved model to disk")

for j in range(10):
    for i in range(int(14000/minibatchsize)):
        train(i)

start
end
250/250 [==============================] - 1s 4ms/step
Epoch:  0 Iter:  0 Metric:  [0.04716492912173271, 0.012473558656871319, 0.034691369652748105]
start
end
250/250 [==============================] - 0s 2ms/step
Epoch:  0 Iter:  1 Metric:  [0.05085961389541626, 0.013732827454805373, 0.03712678557634354]
start
end
250/250 [==============================] - 0s 2ms/step
Epoch:  0 Iter:  2 Metric:  [0.046590393364429473, 0.01240386337786913, 0.034186530321836474]
start
end
250/250 [==============================] - 0s 2ms/step
Epoch:  0 Iter:  3 Metric:  [0.043368083000183105, 0.01156297668069601, 0.031805106341838835]
start
end
250/250 [==============================] - 0s 2ms/step
Epoch:  0 Iter:  4 Metric:  [0.04878781908750534, 0.014503061525523662, 0.03428475758433342]
start
end
250/250 [==============================] - 0s 2ms/step
Epoch:  0 Iter:  5 Metric:  [0.05195894348621368, 0.013650410167872907, 0.03830853286385536]
Saved model to disk
start
end
250/250 [==========

In [0]:
#Check Predictions from Training Data

def eval():
    minibatchsize = 250
    for j in range(1):
    for i in range(int(250/minibatchsize)):
        print("s")
        X_test, Y_test = load_batch_data(i, minibatchsize)
        print('e')
        predict = model.predict(X_test, batch_size=None, verbose=0, steps=None)
        print(np.shape(predict))
        predcoords = predict   
    return predcoords

predcoords = eval()

s
e
(2, 250, 2)
[[0.843415   0.8484043 ]
 [0.4432834  0.6777086 ]
 [0.4249298  0.91915506]
 [0.582076   0.3051185 ]
 [0.28737345 0.590302  ]
 [0.8431313  0.8762958 ]
 [0.8875268  0.8759326 ]
 [0.6806956  0.5728009 ]
 [0.94357026 0.6587767 ]
 [0.4171562  0.79515815]
 [0.39117554 0.47374386]
 [0.9040463  0.6471067 ]
 [0.20069924 0.89887977]
 [0.8558965  0.6779447 ]
 [0.59754205 0.3478883 ]
 [0.80336314 0.6726337 ]
 [0.83095145 0.4377718 ]
 [0.5433287  0.65432364]
 [0.70848054 0.79028183]
 [0.7411434  0.5030624 ]
 [0.7836751  0.7788601 ]
 [0.34661385 0.9353486 ]
 [0.57514226 0.56595594]
 [0.5161083  0.30311823]
 [0.9032663  0.93349934]
 [0.35788405 0.7326567 ]
 [0.5764573  0.57433397]
 [0.88800395 0.42478013]
 [0.41188872 0.80014837]
 [0.352826   0.86464477]
 [0.7242155  0.76327753]
 [0.67806983 0.95371556]
 [0.5949782  0.48198688]
 [0.30992913 0.90189195]
 [0.2722294  0.7037068 ]
 [0.9655327  0.9595418 ]
 [0.5524064  0.36585772]
 [0.72991645 0.39619672]
 [0.35828024 0.84785354]
 [0.76696

In [0]:
#Plot images to show Predictions vs Ground Truth

def plot_image(i):
    img = Image.open(trainfileslist[i])
    fig,ax = plt.subplots(1)
    ax.imshow(img)
    x = (coordinates[i,0]-(coordinates[i,2]/2))*640
    y = (coordinates[i,1]-(coordinates[i,3]/2))*480
    w = (coordinates[i,2])*640
    h = (coordinates[i,3])*480
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
    x = (predcoords[0][i][0]-(predcoords[1][i][0]/2))*640
    y = (predcoords[0][i][1]-(predcoords[1][i][1]/2))*480
    w = (predcoords[1][i][0])*640
    h = (predcoords[1][i][1])*480
    rect = patches.Rectangle((x,y),w,h,linewidth=1,edgecolor='g',facecolor='none')
    ax.add_patch(rect)
    plt.show()
    
plot_image(5)
plot_image(15)
plot_image(25)
plot_image(35)
plot_image(45)
plot_image(55)
plot_image(65)
plot_image(75)
plot_image(85)
plot_image(95)
plot_image(105)
plot_image(115)
plot_image(125)
plot_image(135)
plot_image(145)
plot_image(155)
plot_image(165)
plot_image(175)